In [9]:
from datasets import load_dataset

# Tải tập dữ liệu từ Hugging Face Hub
dataset = load_dataset("opus_books","en-hu")

# Hiển thị một số mẫu dữ liệu từ tập huấn luyện
print(dataset["train"][0:5])


{'id': ['0', '1', '2', '3', '4'], 'translation': [{'en': 'Source: Project GutenbergAudiobook available here', 'hu': 'Source: mek.oszk.huTranslation: Szenczi MiklósAudiobook available here'}, {'en': 'Pride and Prejudice', 'hu': 'Büszkeség és balítélet'}, {'en': 'Jane Austen', 'hu': 'Jane Austen'}, {'en': 'Chapter 1', 'hu': 'I. KÖNYV 1. FEJEZET'}, {'en': 'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.', 'hu': 'Általánosan elismert igazság, hogy a legényembernek, ha vagyonos, okvetlenül kell feleség.'}]}


In [10]:
import pandas as pd
from datasets import load_dataset

# Tải tập dữ liệu từ Hugging Face Hub
dataset = load_dataset("opus_books", "en-hu")

# Chuyển đổi tập dữ liệu thành DataFrame
data = dataset['train'][:]
df = pd.DataFrame(data)

# Tách cột 'translation' thành hai cột riêng biệt 'en' và 'hu'
df = df.join(pd.json_normalize(df['translation']))
df.drop(columns=['translation'], inplace=True)

# Lưu DataFrame thành file CSV
df.to_csv("opus_books_en_hu.csv", index=False)

print("Dữ liệu đã được lưu vào file opus_books_en_hu.csv")


Dữ liệu đã được lưu vào file opus_books_en_hu.csv


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, BertConfig

# Đọc dữ liệu từ file CSV
data = pd.read_csv("test.csv")

# Chia thành dữ liệu huấn luyện, validation và kiểm tra
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Khởi tạo tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Định nghĩa lớp Dataset cho dữ liệu
class TranslationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        en_sentence = self.data.iloc[idx]['en']
        hu_sentence = self.data.iloc[idx]['hu']

        encoding = self.tokenizer(en_sentence, hu_sentence, 
                                  return_tensors='pt', 
                                  max_length=self.max_length, 
                                  padding='max_length', 
                                  truncation=True)
        
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        # Add labels
        labels = input_ids.clone()
        labels[input_ids == self.tokenizer.pad_token_id] = -100  # Mask token
        
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


# Khởi tạo dataset cho huấn luyện, validation và kiểm tra
train_dataset = TranslationDataset(train_data, tokenizer)
val_dataset = TranslationDataset(val_data, tokenizer)
test_dataset = TranslationDataset(test_data, tokenizer)

# Khởi tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

# Xây dựng mô hình Transformer
class TransformerTranslator(nn.Module):
    def __init__(self):
        super(TransformerTranslator, self).__init__()
        self.config = BertConfig.from_pretrained('bert-base-multilingual-cased')
        self.model = BertModel.from_pretrained('bert-base-multilingual-cased', config=self.config)
        self.fc = nn.Linear(self.config.hidden_size, self.config.vocab_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.last_hidden_state)
        return logits

# Huấn luyện mô hình
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerTranslator().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * input_ids.size(0)
    
    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))
            val_loss += loss.item() * input_ids.size(0)
            
            _, predicted = torch.max(outputs, 2)
            correct += (predicted == labels).sum().item()
            total += labels.size(0) * labels.size(1)
    
    val_loss /= len(val_loader.dataset)
    accuracy = correct / total
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}")

# Đánh giá mô hình trên tập kiểm tra
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))
        test_loss += loss.item() * input_ids.size(0)
        
        _, predicted = torch.max(outputs, 2)
        correct += (predicted == labels).sum().item()
        total += labels.size(0) * labels.size(1)

test_loss /= len(test_loader.dataset)
accuracy = correct / total

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")
